In [35]:
# For OpenAI

from dotenv import load_dotenv
import os

load_dotenv()

openai_endpoint = os.getenv("OPENAI_ENDPOINT")
openai_token = os.getenv("OPENAI_TOKEN")
openai_model = os.getenv("OPENAI_MODEL")
openai_api_version = os.getenv("OPENAI_API_VERSION")

import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output


# define LLM
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(
    api_base=openai_endpoint,
    api_key=openai_token,
    temperature=0, 
    model=openai_model,
)
Settings.chunk_size = 512

In [18]:
%pip install llama-index-readers-wikipedia
%pip install llama-index-llms-azure-openai
%pip install llama-index-graph-stores-nebula
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-azure-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install ipython-ngql nebula3-python

  Using cached pyarrow-17.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached pyarrow-17.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (39.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 382.7 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

os.environ["NEBULA_USER"] = "root"
os.environ["NEBULA_PASSWORD"] = "nebula"  # default is "nebula"
os.environ["NEBULA_ADDRESS"] = "127.0.0.1:9669"  # assumed we have NebulaGraph installed locally

space_name = "llamaindex"
edge_types, rel_prop_names = ["relationship"], ["relationship"]  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [19]:
%pip install nebula3-python


Note: you may need to restart the kernel to use updated packages.


In [20]:
# create a nebula space 
from nebula3.Config import Config
from nebula3.gclient.net import ConnectionPool
from nebula3.gclient.net import Session

# Nebula Graph 클러스터 설정
config = Config()
config.max_connection_pool_size = 10

# 네뷸라 그래프 클러스터의 IP 주소와 포트 설정
connection_pool = ConnectionPool()
nebula_cluster_address = [('127.0.0.1', 9669)]  # Nebula Graph 클러스터의 주소와 포트
connection_pool.init(nebula_cluster_address, config)

# Nebula Graph 세션 생성
session = connection_pool.get_session('root', 'nebula')

# 새로운 space 생성 명령어
create_space_query = """
CREATE SPACE IF NOT EXISTS llamaindex(
    vid_type = FIXED_STRING(32),
    partition_num = 10,
    replica_factor = 1
);
USE llamaindex;
"""

# 쿼리 실행
result = session.execute(create_space_query)

# 결과 확인
if result.is_succeeded():
    print("Space 'demo_space' created successfully!")
else:
    print(f"Failed to create space. Error: {result.error_msg()}")

# 세션 종료
session.release()

Space 'demo_space' created successfully!


In [21]:
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore

graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [25]:
%pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=33b57af52e8923d5e3f4b7aac52e2f137bac2c8e1b6ec4105340404cc481db84
  Stored in directory: /home/js/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [27]:
from llama_index.core import download_loader

from llama_index.readers.wikipedia import WikipediaReader

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Guardians of the Galaxy Vol. 3"], auto_suggest=False
)
print(documents)

[Document(id_='51126744', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='Guardians of the Galaxy Vol. 3 (marketed as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the other Guardians race to save the life of Rocket (Cooper) from his creator, the High Evolutionary (Iwuji), who is an alien scientist trying to perfect th

In [36]:
from llama_index.core import KnowledgeGraphIndex

kg_index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

ValueError: Unknown model 'lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF'. Please provide a valid OpenAI model name in: gpt-4, gpt-4-32k, gpt-4-1106-preview, gpt-4-0125-preview, gpt-4-turbo-preview, gpt-4-vision-preview, gpt-4-1106-vision-preview, gpt-4-turbo-2024-04-09, gpt-4-turbo, gpt-4o, gpt-4o-2024-05-13, gpt-4o-2024-08-06, gpt-4o-mini, gpt-4o-mini-2024-07-18, gpt-4-0613, gpt-4-32k-0613, gpt-4-0314, gpt-4-32k-0314, gpt-3.5-turbo, gpt-3.5-turbo-16k, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, gpt-3.5-turbo-0613, gpt-3.5-turbo-16k-0613, gpt-3.5-turbo-0301, text-davinci-003, text-davinci-002, gpt-3.5-turbo-instruct, text-ada-001, text-babbage-001, text-curie-001, ada, babbage, curie, davinci, gpt-35-turbo-16k, gpt-35-turbo, gpt-35-turbo-0125, gpt-35-turbo-1106, gpt-35-turbo-0613, gpt-35-turbo-16k-0613